In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import random
import math

eta = 0.7   # learning rate
epoch = 10001 # iteration


In [2]:
def sigmoid(x):
    return 1.0/(1+ np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)

In [3]:
# Logistic Regression Model
class LogisticRegression:
    
    def __init__(self, x, w, y):
        self.inputs  = x
        self.weights = w               
        self.target  = y
        self.output  = np.zeros(self.target.shape)

    def forward_proc(self):
       # forward processing of inputs and weights using sigmoid activation function 
        self.output = sigmoid(np.dot(self.weights, self.inputs.T))

    def backprop(self):
        # backward processing of appling the chain rule to find derivative of the cross-entropy losss w.r.t weights
        dw = (self.output - self.target) * self.inputs # same formular for both linear and logistic regression

        # update the weights with the derivative of the loss function
        self.weights -= eta * dw

    def predict(self, x):
        # predict the output for a given input x
        return (sigmoid(np.dot(self.weights, x.T)))
        
    def calculate_error(self):
        # calculate error
        error = -self.target * math.log(self.output) - (1-self.target) * math.log(1-self.output)
        return abs(error)

In [6]:
df = pd.read_csv("/content/titanic_data.csv")

df['Sex'] = df['Sex'].map({'female':1,'male':0})
df['Age'].fillna(value=df['Age'].mean(), inplace=True)

df = df.iloc[:100, :]
passengers = df[["Sex", "Age", "Pclass","Survived" ]]

In [7]:
# split train and test set
train, test = train_test_split(passengers, test_size=0.2)

In [8]:
# select proper features for prediction
train_features = train[["Sex", "Age", "Pclass"]].values
test_features = test[["Sex", "Age", "Pclass"]].values
train_labels = train[["Survived"]].values
test_labels = test[["Survived"]].values

scaler = MinMaxScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.fit_transform(test_features)

concat_data = np.concatenate((train_features, train_labels), axis = 1)
np.random.shuffle(concat_data)

weights = np.random.rand(1, 3)
print("Initial Weights:", weights)

Initial Weights: [[0.5928544  0.70830884 0.47865433]]


In [9]:
# SGD Optimization
for i in range(epoch):

    if i == 0: w = weights
    np.random.shuffle(concat_data) # shuffle the input data
    X = concat_data[:, 0:3]
    y = concat_data[:, 3:4]

    for j in range(len(X)):
        model = LogisticRegression(X[j], w, y[j])
        model.forward_proc()
        model.backprop()
        w = model.weights 

    if (i % 1000) == 0:
        print("Loss: ", model.calculate_error())
    
print("Output:", model.output)
print("Adjusted Weights:", model.weights)

Loss:  [0.53892939]
Loss:  [0.3122651]
Loss:  [1.43991073]
Loss:  [0.11784073]
Loss:  [0.39169833]
Loss:  [0.03106825]
Loss:  [0.3001235]
Loss:  [0.19876907]
Loss:  [0.04166055]
Loss:  [1.06058468]
Loss:  [0.08393887]
Output: [0.08051253]
Adjusted Weights: [[ 3.37564528 -1.29950912 -2.10483557]]


In [10]:
# Performance measure
# use the test features
X = test_features
y = test_labels
w = model.weights # use the weights resulting from training
y_predic = []
y_actual = []
for j in range(len(X)):
    model = LogisticRegression(X[j], w, y[j])
    y_actual.append(y[j][0]);
    if model.predict(X[j]) >= 0.5:
        y_predic.append(1)
    elif model.predict(X[j]) < 0.5:
        y_predic.append(0)


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
results = confusion_matrix(y_actual, y_predic)

print ('\n\nConfusion Matrix :')
print(results)
print ('Classification Report : ')
print (classification_report(y_actual, y_predic))



Confusion Matrix :
[[11  2]
 [ 2  5]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        13
           1       0.71      0.71      0.71         7

    accuracy                           0.80        20
   macro avg       0.78      0.78      0.78        20
weighted avg       0.80      0.80      0.80        20

